# Titers for a serum
Analyze titers for a serum, aggregating replicates which may be across multiple plates.

In [1]:
import pickle

import altair as alt

import neutcurve

import pandas as pd

_ = alt.data_transformers.disable_max_rows()

Get variables from `snakemake`:

In [2]:

######## snakemake preamble start (automatically inserted, do not edit) ########
import sys; sys.path.extend(['/fh/fast/bloom_j/software/miniconda3/envs/seqneut-pipeline/lib/python3.11/site-packages', '/fh/fast/bloom_j/computational_notebooks/jbloom/2023/flu_seqneut_DRIVE_2021-22_repeat_vax/seqneut-pipeline', '/fh/fast/bloom_j/computational_notebooks/jbloom/2023/flu_seqneut_DRIVE_2021-22_repeat_vax/seqneut-pipeline/test_example/..', '/fh/fast/bloom_j/computational_notebooks/jbloom/2023/flu_seqneut_DRIVE_2021-22_repeat_vax/seqneut-pipeline/test_example', '/fh/fast/bloom_j/software/miniconda3/envs/seqneut-pipeline/bin', '/fh/fast/bloom_j/software/miniconda3/envs/seqneut-pipeline/lib/python3.11', '/fh/fast/bloom_j/software/miniconda3/envs/seqneut-pipeline/lib/python3.11/lib-dynload', '/home/jbloom/.local/lib/python3.11/site-packages', '/fh/fast/bloom_j/software/miniconda3/envs/seqneut-pipeline/lib/python3.11/site-packages', '/home/jbloom/.cache/snakemake/snakemake/source-cache/runtime-cache/tmpbw69qygs/file/fh/fast/bloom_j/computational_notebooks/jbloom/2023/flu_seqneut_DRIVE_2021-22_repeat_vax/seqneut-pipeline/notebooks', '/fh/fast/bloom_j/computational_notebooks/jbloom/2023/flu_seqneut_DRIVE_2021-22_repeat_vax/seqneut-pipeline/notebooks']); import pickle; snakemake = pickle.loads(b'\x80\x04\x95\xdb\x13\x00\x00\x00\x00\x00\x00\x8c\x10snakemake.script\x94\x8c\tSnakemake\x94\x93\x94)\x81\x94}\x94(\x8c\x05input\x94\x8c\x0csnakemake.io\x94\x8c\nInputFiles\x94\x93\x94)\x81\x94\x8c\'results/plates/plate11/curvefits.pickle\x94a}\x94(\x8c\x06_names\x94}\x94\x8c\x07pickles\x94K\x00K\x01\x86\x94s\x8c\x12_allowed_overrides\x94]\x94(\x8c\x05index\x94\x8c\x04sort\x94eh\x12\x8c\tfunctools\x94\x8c\x07partial\x94\x93\x94h\x06\x8c\x19Namedlist._used_attribute\x94\x93\x94\x85\x94R\x94(h\x18)}\x94\x8c\x05_name\x94h\x12sNt\x94bh\x13h\x16h\x18\x85\x94R\x94(h\x18)}\x94h\x1ch\x13sNt\x94bh\x0eh\x06\x8c\tNamedlist\x94\x93\x94)\x81\x94h\na}\x94(h\x0c}\x94h\x10]\x94(h\x12h\x13eh\x12h\x16h\x18\x85\x94R\x94(h\x18)}\x94h\x1ch\x12sNt\x94bh\x13h\x16h\x18\x85\x94R\x94(h\x18)}\x94h\x1ch\x13sNt\x94bubub\x8c\x06output\x94h\x06\x8c\x0bOutputFiles\x94\x93\x94)\x81\x94(\x8c-results/sera/Y044d30/titers_per_replicate.csv\x94\x8c\x1fresults/sera/Y044d30/titers.csv\x94\x8c\x1fresults/sera/Y044d30/curves.pdf\x94\x8c%results/sera/Y044d30/curvefits.pickle\x94\x8c!results/sera/Y044d30/qc_drops.yml\x94e}\x94(h\x0c}\x94(\x8c\x0eper_rep_titers\x94K\x00N\x86\x94\x8c\x06titers\x94K\x01N\x86\x94\x8c\ncurves_pdf\x94K\x02N\x86\x94\x8c\x06pickle\x94K\x03N\x86\x94\x8c\x08qc_drops\x94K\x04N\x86\x94uh\x10]\x94(h\x12h\x13eh\x12h\x16h\x18\x85\x94R\x94(h\x18)}\x94h\x1ch\x12sNt\x94bh\x13h\x16h\x18\x85\x94R\x94(h\x18)}\x94h\x1ch\x13sNt\x94bh;h4h=h5h?h6hAh7hCh8ub\x8c\x06params\x94h\x06\x8c\x06Params\x94\x93\x94)\x81\x94(]\x94(\x8c\x14A/California/07/2009\x94\x8c\x12A/Michigan/45/2015\x94\x8c\x12A/Brisbane/02/2018\x94\x8c\x11A/Ghana/2080/2020\x94\x8c\x18A/Cote_DIvoire/1448/2021\x94\x8c\x10A/Ghana/138/2020\x94\x8c\x0fA/Togo/845/2020\x94\x8c\x10A/Togo/0274/2021\x94\x8c\x10A/Hawaii/70/2019\x94\x8c\x12A/Niger/10217/2021\x94\x8c\x19A/SouthAfrica/R16462/2021\x94\x8c\x10A/Togo/0304/2021\x94\x8c\x12A/Paris/31196/2021\x94\x8c\x14A/Belgium/H0017/2022\x94\x8c\x18A/England/220200318/2022\x94\x8c\x12A/Paris/30353/2021\x94\x8c\x10A/Nimes/871/2021\x94\x8c\x14A/Belgium/H0038/2022\x94\x8c\x14A/Wisconsin/588/2019\x94\x8c\x14A/Washington/23/2020\x94\x8c\x1aA/India/Pun-NIV312851/2021\x94\x8c\x1aA/India-PUN-NIV328484/2021\x94\x8c\x16A/Bangladesh/8036/2021\x94\x8c\x0eA/Perth/1/2022\x94\x8c\x19A/SouthAfrica/R14850/2021\x94\x8c\x13A/Norway/25089/2022\x94\x8c\x1cA/Bangladesh/3210810034/2021\x94\x8c\x13A/Chester/5355/2022\x94\x8c\x12A/Michigan/19/2021\x94\x8c\x0eA/Utah/27/2022\x94\x8c\x12A/Newcastle/2/2022\x94\x8c\x1fA/India-Pune-Nivcov2221170/2022\x94\x8c\x10A/Sydney/43/2022\x94\x8c\x12A/Brisbane/48/2022\x94\x8c\x16A/Bangladesh/8002/2021\x94\x8c\x16A/Bangladesh/2221/2021\x94e\x8c\x08midpoint\x94}\x94(\x8c\x0emin_replicates\x94K\x02\x8c\x1bmax_fold_change_from_median\x94K\x03\x8c\tignore_qc\x94\x89ue}\x94(h\x0c}\x94(\x8c\x17viral_strain_plot_order\x94K\x00N\x86\x94\x8c\x0eserum_titer_as\x94K\x01N\x86\x94\x8c\rqc_thresholds\x94K\x02N\x86\x94uh\x10]\x94(h\x12h\x13eh\x12h\x16h\x18\x85\x94R\x94(h\x18)}\x94h\x1ch\x12sNt\x94bh\x13h\x16h\x18\x85\x94R\x94(h\x18)}\x94h\x1ch\x13sNt\x94bh~hRh\x80hwh\x82hxub\x8c\twildcards\x94h\x06\x8c\tWildcards\x94\x93\x94)\x81\x94\x8c\x07Y044d30\x94a}\x94(h\x0c}\x94\x8c\x05serum\x94K\x00N\x86\x94sh\x10]\x94(h\x12h\x13eh\x12h\x16h\x18\x85\x94R\x94(h\x18)}\x94h\x1ch\x12sNt\x94bh\x13h\x16h\x18\x85\x94R\x94(h\x18)}\x94h\x1ch\x13sNt\x94b\x8c\x05serum\x94h\x91ub\x8c\x07threads\x94K\x01\x8c\tresources\x94h\x06\x8c\tResources\x94\x93\x94)\x81\x94(K\x01K\x01\x8c\x04/tmp\x94e}\x94(h\x0c}\x94(\x8c\x06_cores\x94K\x00N\x86\x94\x8c\x06_nodes\x94K\x01N\x86\x94\x8c\x06tmpdir\x94K\x02N\x86\x94uh\x10]\x94(h\x12h\x13eh\x12h\x16h\x18\x85\x94R\x94(h\x18)}\x94h\x1ch\x12sNt\x94bh\x13h\x16h\x18\x85\x94R\x94(h\x18)}\x94h\x1ch\x13sNt\x94bh\xa8K\x01h\xaaK\x01h\xach\xa5ub\x8c\x03log\x94h\x06\x8c\x03Log\x94\x93\x94)\x81\x94\x8c)results/sera/Y044d30/Y044d30_titers.ipynb\x94a}\x94(h\x0c}\x94\x8c\x08notebook\x94K\x00N\x86\x94sh\x10]\x94(h\x12h\x13eh\x12h\x16h\x18\x85\x94R\x94(h\x18)}\x94h\x1ch\x12sNt\x94bh\x13h\x16h\x18\x85\x94R\x94(h\x18)}\x94h\x1ch\x13sNt\x94bh\xbeh\xbbub\x8c\x06config\x94}\x94(\x8c\x10seqneut-pipeline\x94\x8c\x03../\x94\x8c\x04docs\x94\x8c\x07../docs\x94\x8c\x0bdescription\x94X\xba\x01\x00\x00# Test example for [seqneut-pipeline](https://github.com/jbloomlab/seqneut-pipeline)\nThis is a small toy-example created by subsetting a real experiment dataset.\n\nSee [https://github.com/jbloomlab/seqneut-pipeline](https://github.com/jbloomlab/seqneut-pipeline)\nfor the computer code and underlying numerical data.\n\nSee [here](https://github.com/jbloomlab/seqneut-pipeline/graphs/contributors) for a\nlist of all contributors to the pipeline.\n\x94\x8c\x0fviral_libraries\x94}\x94\x8c\x14pdmH1N1_lib2023_loes\x94\x8c-data/viral_libraries/pdmH1N1_lib2023_loes.csv\x94s\x8c\x17viral_strain_plot_order\x94\x8c data/viral_strain_plot_order.csv\x94\x8c\x12neut_standard_sets\x94}\x94\x8c\x08loes2023\x94\x8c3data/neut_standard_sets/loes2023_neut_standards.csv\x94s\x8c\x1eillumina_barcode_parser_params\x94}\x94(\x8c\x08upstream\x94\x8c\x1fCTCCCTACAATGTCGGATTTGTATTTAATAG\x94\x8c\ndownstream\x94\x8c\x00\x94\x8c\x04minq\x94K\x14\x8c\x11upstream_mismatch\x94K\x04\x8c\x0ebc_orientation\x94\x8c\x02R2\x94u\x8c#default_process_plate_qc_thresholds\x94}\x94(\x8c\x1bavg_barcode_counts_per_well\x94K\xfa\x8c\x1fmin_neut_standard_frac_per_well\x94G?tz\xe1G\xae\x14{\x8c"no_serum_per_viral_barcode_filters\x94}\x94(\x8c\x08min_frac\x94G?@bM\xd2\xf1\xa9\xfc\x8c\x0fmax_fold_change\x94K\x03\x8c\tmax_wells\x94K\x02u\x8c!per_neut_standard_barcode_filters\x94}\x94(\x8c\x08min_frac\x94G?tz\xe1G\xae\x14{\x8c\x0fmax_fold_change\x94K\x03\x8c\tmax_wells\x94K\x02u\x8c min_neut_standard_count_per_well\x94M\xf4\x01\x8c)min_no_serum_count_per_viral_barcode_well\x94K\x1e\x8c+max_frac_infectivity_per_viral_barcode_well\x94K\x05\x8c)min_dilutions_per_barcode_serum_replicate\x94K\x06u\x8c%default_process_plate_curvefit_params\x94}\x94(\x8c\x18frac_infectivity_ceiling\x94K\x01\x8c\x06fixtop\x94\x89\x8c\tfixbottom\x94K\x00u\x8c!default_process_plate_curvefit_qc\x94}\x94(\x8c\x1dmax_frac_infectivity_at_least\x94G?\xe0\x00\x00\x00\x00\x00\x00\x8c\x06min_R2\x94G?\xe9\x99\x99\x99\x99\x99\x9a\x8c#serum_replicates_ignore_curvefit_qc\x94]\x94\x8c+barcode_serum_replicates_ignore_curvefit_qc\x94]\x94u\x8c\x06plates\x94}\x94(\x8c\x06plate2\x94}\x94(\x8c\x04date\x94\x8c\x08datetime\x94\x8c\x04date\x94\x93\x94C\x04\x07\xe7\x08\x01\x94\x85\x94R\x94\x8c\rviral_library\x94\x8c\x14pdmH1N1_lib2023_loes\x94\x8c\x11neut_standard_set\x94\x8c\x08loes2023\x94\x8c\x0bsamples_csv\x94\x8c\x1edata/plates/plate2_samples.csv\x94\x8c\x0cmanual_drops\x94}\x94\x8c\rqc_thresholds\x94}\x94(h\xe7K\xfah\xe8G?tz\xe1G\xae\x14{h\xe9}\x94(h\xebG?@bM\xd2\xf1\xa9\xfch\xecK\x03h\xedK\x02uh\xee}\x94(h\xf0G?tz\xe1G\xae\x14{h\xf1K\x03h\xf2K\x02uh\xf3M\xf4\x01h\xf4K\x1eh\xf5K\x05h\xf6K\x06u\x8c\x0fcurvefit_params\x94}\x94(h\xf9K\x01h\xfa\x89h\xfbK\x00u\x8c\x0bcurvefit_qc\x94}\x94(h\xfeG?\xe0\x00\x00\x00\x00\x00\x00h\xffG?\xe9\x99\x99\x99\x99\x99\x9aj\x00\x01\x00\x00j\x01\x01\x00\x00j\x02\x01\x00\x00j\x03\x01\x00\x00uu\x8c\x07plate11\x94}\x94(\x8c\x04date\x94j\x0b\x01\x00\x00C\x04\x07\xe7\t\x1a\x94\x85\x94R\x94\x8c\rviral_library\x94\x8c\x14pdmH1N1_lib2023_loes\x94\x8c\x11neut_standard_set\x94\x8c\x08loes2023\x94\x8c\x0bsamples_csv\x94\x8c\x1fdata/plates/plate11_samples.csv\x94\x8c\x0cmanual_drops\x94}\x94\x8c\rqc_thresholds\x94}\x94(h\xe7K\xfah\xe8G?tz\xe1G\xae\x14{h\xe9}\x94(h\xebG?@bM\xd2\xf1\xa9\xfch\xecK\x03h\xedK\x02uh\xee}\x94(h\xf0G?tz\xe1G\xae\x14{h\xf1K\x03h\xf2K\x02uh\xf3M\xf4\x01h\xf4K\x1eh\xf5K\x05h\xf6K\x06u\x8c\x0fcurvefit_params\x94}\x94(h\xf9K\x01h\xfa\x89h\xfbK\x00u\x8c\x0bcurvefit_qc\x94}\x94(h\xfeG?\xe0\x00\x00\x00\x00\x00\x00h\xffG?\xe9\x99\x99\x99\x99\x99\x9aj\x00\x01\x00\x00j\x01\x01\x00\x00j\x02\x01\x00\x00]\x94]\x94(\x8c\x10AGGTCAAGACCACAGG\x94\x8c\x06M099d0\x94eauuu\x8c\x16default_serum_titer_as\x94hw\x8c\x1bdefault_serum_qc_thresholds\x94hx\x8c\x16sera_override_defaults\x94}\x94u\x8c\x04rule\x94\x8c\x0cserum_titers\x94\x8c\x0fbench_iteration\x94N\x8c\tscriptdir\x94\x8ct/fh/fast/bloom_j/computational_notebooks/jbloom/2023/flu_seqneut_DRIVE_2021-22_repeat_vax/seqneut-pipeline/notebooks\x94ub.'); from snakemake.logging import logger; logger.printshellcmds = False; import os; os.chdir(r'/fh/fast/bloom_j/computational_notebooks/jbloom/2023/flu_seqneut_DRIVE_2021-22_repeat_vax/seqneut-pipeline/test_example');
######## snakemake preamble end #########


In [3]:
pickle_fits = snakemake.input.pickles
per_rep_titers_csv = snakemake.output.per_rep_titers
titers_csv = snakemake.output.titers
curves_pdf = snakemake.output.curves_pdf
output_pickle = snakemake.output.pickle
qc_drops_file = snakemake.output.qc_drops
viral_strain_plot_order = snakemake.params.viral_strain_plot_order
serum_titer_as = snakemake.params.serum_titer_as
qc_thresholds = snakemake.params.qc_thresholds
serum = snakemake.wildcards.serum

print(f"Processing {serum=}")

Processing serum='Y044d30'


Combine all the pickled `neutcurve.CurveFits` from plates for this serum into a single `neutcurve.CurveFits`:

In [5]:
print(f"Combining the curve fits for {serum=} from {pickle_fits=}")

fits_to_combine = []
for fname in pickle_fits:
    with open(fname, "rb") as f:
        fits_to_combine.append(pickle.load(f))
fits_noqc = neutcurve.CurveFits.combineCurveFits(fits_to_combine, sera=[serum])

Combining the curve fits for serum='Y044d30' from pickle_fits=['results/plates/plate11/curvefits.pickle']


Indicate how we are calculating the titer:

In [23]:
print(f"Calculating with {serum_titer_as=}")
assert serum_titer_as in {"nt50", "midpoint"}

Calculating with serum_titer_as='midpoint'


Get all the fit params with the titers, initially without any QC.
We also convert the IC50 to NT50, and take inverse of midpoint to get it on same scale as NT50s:

In [22]:
def get_median_bound(s):
    """Get the bound on titer when taking median."""
    s = list(s)
    if len(s) % 2:
        return s[len(s) // 2]
    else:
        bounds = s[len(s) // 2 - 1 : len(s) // 2 + 1]
        assert len(bounds) == 2
        if len(set(bounds)) == 1:
            return bounds[0]
        elif "interpolated" in bounds:
            return [b for b in bounds if b != "interpolated"][0]
        else:
            return "inconsistent"

fit_params_noqc = (
    fits_noqc.fitParams(average_only=False, no_average=True)
    .assign(
        nt50=lambda x: 1 / x["ic50"],
        midpoint=lambda x: 1 / x["midpoint_bound"],
        titer=lambda x: x["midpoint"] if serum_titer_as == "midpoint" else x["nt50"],
        titer_bound=lambda x: (
            x["midpoint_bound_type"] if serum_titer_as == "midpoint" else x["ic50_bound"]
        ).map({"lower": "upper", "upper": "lower", "interpolated": "interpolated"}),
        titer_as=serum_titer_as,
    )
    [["serum", "virus", "replicate", "titer", "titer_bound", "titer_as", "nt50", "midpoint", "top", "bottom"]]
)
assert fit_params_noqc.notnull().all().all()

if len(invalid_titer_as := fit_params_noqc.query("(titer_as == 'nt50') and top <= 0.5")):
    raise ValueError(
        f"There are titers computed as nt50 when curve top <= 0.5:\n{invalid_titer_as}"
    )

Plot the correlation of the NT50s with the midpoint (this is an interactive plot, mouse over points for details).
This plot can help you determine if you made the correct choice of `serum_titer_as` when choosing to use the midpoint or NT50 for the titer.
For titers where they are well correlated it should not matter which you chose.
But if there are titers far from the correlation line, you should look at those measurements and curves to make sure you made the correct choice of calculating the titer as the NT50 versus midpoint:

Plot the individual neutralization curves:

fig, _ = serum_curvefits.plotReplicates(
    attempt_shared_legend=False,
    legendfontsize=8,
    ncol=4,
    heightscale=1.25,
    widthscale=1.25,
    viruses=viruses,
    subplot_titles="{virus}",
)

_ = fig.suptitle(
    f"neutralization curves for {serum}", y=1, fontsize=18, fontweight="bold"
)

fig.tight_layout()

Save the curves to a file:

print(f"Saving to {curves_pdf}")
fig.savefig(curves_pdf)

Save the `CurveFits` for the serum to a pickle file:

with open(output_pickle, "wb") as f:
    pickle.dump(serum_curvefits, f)